# PySpark Huggingface Inferencing
### Text Classification using Pipelines with Tensorflow

Based on: https://huggingface.co/docs/transformers/quicktour#pipeline-usage

In [1]:
import tensorflow as tf
from transformers import pipeline

2024-09-25 17:33:36.783509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 17:33:36.800587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 17:33:36.805800: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 17:33:36.819189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 17:33:37.652146: W tensorflow/comp

In [2]:
# set device if tensorflow gpu is available
device = 0 if tf.config.list_physical_devices('GPU') else -1

In [3]:
print(tf.__version__)

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2.17.0


In [4]:
classifier = pipeline("sentiment-analysis", device=device)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
2024-09-25 17:33:42.546470: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30099 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB-H, pci bus id: 0000:34:00.0, compute capability: 7.0
2024-09-25 17:33:42.547912: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31135 MB memory:  -> device: 1, name: Tesla V100-SXM3-32GB-H, pci bus id: 0000:36:00.0, compute capability: 7.0
2024-09-25 17:33:42.549203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31135 MB memory:  -> device: 

In [5]:
classifier(("We are very happy to show you the 🤗 Transformers library."))

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [6]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


#### Use another model and tokenizer in the pipeline

In [7]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

In [8]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [9]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

[{'label': '5 stars', 'score': 0.7272652983665466}]

## Inference using Spark DL API

In [10]:
import pandas as pd
from pyspark.sql.functions import col, struct, pandas_udf
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import FloatType, StringType, StructField, StructType
from pyspark.sql import SparkSession

In [11]:
num_threads = 6

# Creating a local Spark session for demonstration, in case it hasn't already been created.

_config = {
    "spark.master": f"local[{num_threads}]",
    "spark.driver.host": "127.0.0.1",
    "spark.task.maxFailures": "1",
    "spark.driver.memory": "8g",
    "spark.executor.memory": "8g",
    "spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled": "false",
    "spark.sql.pyspark.jvmStacktrace.enabled": "true",
    "spark.sql.execution.arrow.pyspark.enabled": "true",
    "spark.python.worker.reuse": "true",
}
spark = SparkSession.builder.appName("spark-dl-example")
for key, value in _config.items():
    spark = spark.config(key, value)
spark = spark.getOrCreate()

sc = spark.sparkContext

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
24/09/25 17:33:48 WARN Utils: Your hostname, dgx2h0194.spark.sjc4.nvmetal.net resolves to a loopback address: 127.0.1.1; using 10.150.30.2 instead (on interface enp134s0f0np0)
24/09/25 17:33:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 17:33:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
from datasets import load_dataset

# Load the IMDB dataset
data = load_dataset("imdb", split="test")

lines = []
for example in data:
    # first sentence only
    lines.append([example["text"]])

len(lines)

df = spark.createDataFrame(lines, ['lines']).repartition(10).cache()

In [13]:
df.write.mode("overwrite").parquet("imdb_test")

24/09/25 17:33:56 WARN TaskSetManager: Stage 0 contains a task of very large size (5123 KiB). The maximum recommended task size is 1000 KiB.


In [14]:
# only use first sentence of IMDB reviews
@pandas_udf("string")
def first_sentence(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

df = spark.read.parquet("imdb_test").withColumn("sentence", first_sentence(col("lines"))).select("sentence").limit(100).cache()
df.show(truncate=80)

+--------------------------------------------------------------------------------+
|                                                                        sentence|
+--------------------------------------------------------------------------------+
|In following Dylan Moran's star from the charming misanthrope bookstore owner...|
|Here in Australia Nights in Rodanthe is being promoted in the same class as t...|
|           The Tender Hook, or, Who Killed The Australian Film Industry? Case No|
|The only reason I'm even giving this movie a 4 is because it was made in to a...|
|Hooray for Title Misspellings! After reading reviews and contemplating, my gi...|
|                        This movie makes you wish imdb would let you vote a zero|
|To me this film is just a very very lame teen party movie with all the normal...|
|                                                                         I tried|
|Awkward disaster mishmash has a team of scavengers coming across the overturn...|
|Acc

In [15]:
def predict_batch_fn():
    import tensorflow as tf
    from transformers import pipeline

    # Enable GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)
    
    device = 0 if tf.config.list_physical_devices('GPU') else -1
    pipe = pipeline("sentiment-analysis", device=device)
    def predict(inputs):
        return pipe(inputs.tolist())
    return predict

In [16]:
classify = predict_batch_udf(predict_batch_fn,
                             return_type=StructType([
                                 StructField("label", StringType(), True),
                                 StructField("score", FloatType(), True)
                             ]),
                             batch_size=10)

In [17]:
%%time
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify(struct("sentence"))).select("sentence", "preds.*")
results = preds.collect()

2024-09-25 17:34:01.869946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 17:34:01.886921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 17:34:01.891972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 17:34:01.904245: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 17:34:02.586399: W tensorflow/comp

CPU times: user 48.7 ms, sys: 60.1 ms, total: 109 ms
Wall time: 14.9 s


In [18]:
%%time
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify("sentence")).select("sentence", "preds.*")
results = preds.collect()

CPU times: user 33.7 ms, sys: 7.42 ms, total: 41.2 ms
Wall time: 6.99 s


In [19]:
%%time
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify(col("sentence"))).select("sentence", "preds.*")
results = preds.collect()

CPU times: user 24.4 ms, sys: 18.5 ms, total: 42.8 ms
Wall time: 7.02 s


In [20]:
preds.show(truncate=80)

+--------------------------------------------------------------------------------+--------+----------+
|                                                                        sentence|   label|     score|
+--------------------------------------------------------------------------------+--------+----------+
|In following Dylan Moran's star from the charming misanthrope bookstore owner...|NEGATIVE| 0.9464624|
|Here in Australia Nights in Rodanthe is being promoted in the same class as t...|NEGATIVE| 0.9811111|
|           The Tender Hook, or, Who Killed The Australian Film Industry? Case No|NEGATIVE|0.98526716|
|The only reason I'm even giving this movie a 4 is because it was made in to a...|NEGATIVE| 0.9995627|
|Hooray for Title Misspellings! After reading reviews and contemplating, my gi...|NEGATIVE|0.99946946|
|                        This movie makes you wish imdb would let you vote a zero|NEGATIVE| 0.9981305|
|To me this film is just a very very lame teen party movie with all the n

### Using Triton Inference Server

Note: you can restart the kernel and run from this point to simulate running in a different node or environment.  
While the examples above use Tensorflow, note that inference on the Triton server is run using PyTorch. PyTorch will be included in the tarball below, and no further environment changes are required by the user.

This notebook uses the [Python backend with a custom execution environment](https://github.com/triton-inference-server/python_backend#creating-custom-execution-environments) with the compatible versions of Python/Numpy for Triton 24.08, using a conda-pack environment created as follows:
```
conda create -n huggingface -c conda-forge python=3.10.0
conda activate huggingface

export PYTHONNOUSERSITE=True
pip install numpy==1.26.4 conda-pack sentencepiece sentence_transformers transformers

conda-pack  # huggingface.tar.gz
```

In [21]:
import numpy as np
import pandas as pd
import os
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import col, struct, pandas_udf
from pyspark.sql.types import FloatType, StringType, StructField, StructType

In [22]:
%%bash
# copy custom model to expected layout for Triton
rm -rf models
mkdir -p models
cp -r models_config/hf_pipeline models

# add custom execution environment
cp huggingface.tar.gz models

#### Start Triton Server on each executor

In [23]:
num_executors = 1
triton_models_dir = "{}/models".format(os.getcwd())
huggingface_cache_dir = "{}/.cache/huggingface".format(os.path.expanduser('~'))
nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        container=client.containers.run(
            "nvcr.io/nvidia/tritonserver:24.08-py3", "tritonserver --model-repository=/models",
            detach=True,
            device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
            environment=[
                "TRANSFORMERS_CACHE=/cache"
            ],
            name="spark-triton",
            network_mode="host",
            remove=True,
            shm_size="256M",
            volumes={
                triton_models_dir: {"bind": "/models", "mode": "ro"},
                huggingface_cache_dir: {"bind": "/cache", "mode": "rw"}
            }
        )
        print(">>>> starting triton: {}".format(container.short_id))
        # wait for triton to be running
        time.sleep(15)
        
        client = grpcclient.InferenceServerClient("localhost:8001")
        
        elapsed = 0
        timeout = 120
        ready = False
        while not ready and elapsed < timeout:
            try:
                time.sleep(5)
                elapsed += 5
                ready = client.is_server_ready()
            except Exception as e:
                pass

    return [True]

nodeRDD.barrier().mapPartitions(start_triton).collect()

>>>> containers: ['743900a7a117']


[True]

#### Run inference

In [24]:
# only use first sentence of IMDB reviews
@pandas_udf("string")
def first_sentence(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

df = spark.read.parquet("imdb_test").withColumn("sentence", first_sentence(col("lines"))).select("sentence").limit(1000)

In [25]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool_),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
    return predict

In [26]:
from functools import partial

classify = predict_batch_udf(partial(triton_fn, triton_uri="localhost:8001", model_name="hf_pipeline"),
                             return_type=StructType([
                                 StructField("label", StringType(), True),
                                 StructField("score", FloatType(), True)
                             ]),
                             input_tensor_shapes=[[1]],
                             batch_size=100)

In [27]:
%%time
# first pass caches model/fn
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify(struct("sentence"))).select("sentence", "preds.*")
results = preds.collect()

CPU times: user 15.9 ms, sys: 32 ms, total: 47.9 ms
Wall time: 5.74 s


In [28]:
%%time
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify("sentence")).select("sentence", "preds.*")
results = preds.collect()

CPU times: user 25.5 ms, sys: 22.5 ms, total: 48 ms
Wall time: 5.85 s


In [29]:
%%time
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify(col("sentence"))).select("sentence", "preds.*")
results = preds.collect()

CPU times: user 23.4 ms, sys: 21.8 ms, total: 45.2 ms
Wall time: 5.67 s


In [30]:
preds.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+----------+
|sentence                                                                                                                                                                                                                                                                                                |label   |score     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+----------+
|In following Dylan Moran's star from the c

#### Stop Triton Server on each executor

In [31]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.barrier().mapPartitions(stop_triton).collect()

>>>> stopping containers: ['743900a7a117']


[True]

In [32]:
spark.stop()